<center><h1>Hyperparameter Tuning using HyperDrive</h1></center>

### Import Dependencies.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn

from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform

from azureml.data.dataset_factory import TabularDatasetFactory #Lo usamos?
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import os
import json

### Set up Workspace

In [ ]:
ws = Workspace.from_config()

experiment_name = 'HotelBookingDemand'

experiment=Experiment(ws, experiment_name)

In [ ]:
cpu_cluster_name = "udacity-casptone-project"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

### Hyperdrive Configuration


In [ ]:
# Early termination policy. 
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1) 

# Parameter sampler
param_sampling = RandomParameterSampling(
    {
    'C': uniform(0.01, 100),
    'max_iter': choice(100, 1000, 10000)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")
    
azureml_pip_packages = [
    'azureml-defaults', 'azureml-contrib-interpret', 'azureml-telemetry', 'azureml-interpret'
]
  

#Estimator
estimator = SKLearn(entry_script='./train.py', source_directory=".", compute_target=cpu_cluster, pip_packages=azureml_pip_packages)


# Hyperdrive config
hyperdrive_run_config = HyperDriveConfig(estimator=est,
                                         hyperparameter_sampling=ps,
                                         policy=policy,
                                         primary_metric_name='Accuracy',
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=4,
                                         max_concurrent_runs=4
                                        )

In [ ]:
#Experiment Submit
run = experiment.submit(config=hyperdrive_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

In [ ]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = run.get_best_run_by_primary_metric()

model = best_run.register_model(model_name='HotelBookingDemand-lr', 
                                model_path='./outputs/modelht.pkl', 
                                model_framework=Model.Framework.SCIKITLEARN, 
                                model_framework_version='0.22.2',
                                resource_configuration=ResourceConfiguration(cpu=2, memory_in_gb=4),
                                description='Will the hotel booking be canceled?.',
                                tags={'area': 'hospitality', 'type': 'classification'})

print(best_run.get_metrics())

print(model)

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
service_name = 'HotelBookingDemand-HT'
service = Model.deploy(ws, service_name, [model]) #No-code model deployment or Use Code?

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
input_payload = json.dumps({
     "data": [
     {
          "hotel": "Resort Hotel",
          "lead_time": 85,
          "stays_in_weekend_nights": 0,
          "stays_in_week_nights": 3,
          "adults": 2,
          "children": 0,
          "babies": 0,
          "meal": "BB",
          "country": "PRT",
          "market_segment": "Online TA",
          "distribution_channel": "TA/TO",
          "is_repeated_guest": 0,
          "booking_changes": 0,
          "deposit_type": "No Deposit",
          "agent": "240",
          "company": "NULL",
          "days_in_waiting_list": 0,
          "customer_type": "Trasient",
          "adr": 82,
          "required_car_parking_spaces": 1,
          "total_of_special_requests": 0,
          "net_cancelled": 0,
          "Room": 1
     }
   ]
})

output = service.run(input_payload)

print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#service.delete()